In [2]:
!pip install datasets transformers wandb sentencepiece tqdm

     |████████████████████████████████| 325 kB 17.4 MB/s 
     |████████████████████████████████| 4.0 MB 58.4 MB/s 
     |████████████████████████████████| 1.8 MB 53.7 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
     |████████████████████████████████| 212 kB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 49.8 MB/s 
     |████████████████████████████████| 136 kB 64.7 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
     |████████████████████████████████| 127 kB 65.5 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 69.7 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 181 kB 60.6 MB/s 
     |████████████████████████████████| 144 kB 61.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 271 kB 82.1 MB/s 
     |██████████████

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from datasets import load_dataset
from functools import partial

import random
import logging
from tqdm.auto import tqdm
from torch.utils.data import DataLoader


import wandb
import transformers

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small', force_download=True)

Downloading: 100%|██████████| 1.53k/1.53k [00:00<00:00, 1.56MB/s]
Downloading: 100%|██████████| 231M/231M [00:17<00:00, 14.0MB/s] 


In [6]:
dataset = load_dataset('spider')

Reusing dataset spider (C:\Users\wasii\.cache\huggingface\datasets\spider\spider\1.0.0\79778ebea87c59b19411f1eb3eda317e9dd5f7788a556d837ef25c3ae6e5e8b7)
100%|██████████| 2/2 [00:00<00:00, 500.01it/s]


In [46]:
wikisql_dataset = load_dataset('wikisql')

Using custom data configuration default


Dataset wiki_sql downloaded and prepared to C:\Users\wasii\.cache\huggingface\datasets\wiki_sql\default\0.1.0\7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 31.86it/s]


In [57]:
wikisql_dataset['train']['sql']

[{'human_readable': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA',
  'sel': 5,
  'agg': 0,
  'conds': {'column_index': [3],
   'operator_index': [0],
   'condition': ['SOUTH AUSTRALIA']}},
 {'human_readable': 'SELECT Current series FROM table WHERE Notes = New series began in June 2011',
  'sel': 4,
  'agg': 0,
  'conds': {'column_index': [5],
   'operator_index': [0],
   'condition': ['New series began in June 2011']}},
 {'human_readable': 'SELECT Format FROM table WHERE State/territory = South Australia',
  'sel': 2,
  'agg': 0,
  'conds': {'column_index': [0],
   'operator_index': [0],
   'condition': ['South Australia']}},
 {'human_readable': 'SELECT Text/background colour FROM table WHERE State/territory = Australian Capital Territory',
  'sel': 1,
  'agg': 0,
  'conds': {'column_index': [0],
   'operator_index': [0],
   'condition': ['Australian Capital Territory']}},
 {'human_readable': 'SELECT COUNT Fleet Series (Quantity) FROM table WHERE Fuel Propulsion = C

In [54]:
wikisql_dataset['test']['sql'][0]

{'human_readable': 'SELECT Nationality FROM table WHERE Player = Terrence Ross',
 'sel': 2,
 'agg': 0,
 'conds': {'column_index': [0],
  'operator_index': [0],
  'condition': ['Terrence Ross']}}

In [7]:
def preprocess_function(examples, tokenizer, max_seq_length):
    

    inputs = examples['question']
    targets = examples['query']
    
    model_inputs = tokenizer(inputs, max_length=max_seq_length, padding="max_length", truncation=True)
    target_ids = tokenizer(targets, max_length=max_seq_length, padding="max_length", truncation=True)
    target_ids = target_ids.input_ids
    
    #decoder_input_ids = []

    # for target in target_ids:
    #     decoder_input_ids.append([tokenizer.bos_token_id] + target)
    #     labels.append(target + [tokenizer.eos_token_id])

    # model_inputs["decoder_input_ids"] = decoder_input_ids

    labels_with_ignore_index = []
    
    for labels_example in target_ids:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs


In [39]:
max_seq_length=128
overwrite_cache=True
preprocessing_num_workers = 8
batch_size=32
num_train_epochs=5
device='cpu'
learning_rate=1e-5
weight_decay=0.01
lr_scheduler_type = 'linear'
num_warmup_steps = 0
max_train_steps = 20000
logging_steps=25
eval_every_step=1

In [9]:
dataset["train"].column_names

['db_id',
 'query',
 'question',
 'query_toks',
 'query_toks_no_value',
 'question_toks']

In [10]:
column_names = [c for c in dataset["train"].column_names if c not in ['query', 'db_id']]

preprocess_function_wrapped = partial(
    preprocess_function,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
)


processed_datasets = dataset.map(
    preprocess_function_wrapped,
    batched=True,
    num_proc=preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=not overwrite_cache,
    desc="Running tokenizer on dataset",
)

In [11]:
processed_datasets.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"] if "validation" in processed_datasets else processed_datasets["test"]

# Log a few random samples from the training set:
for index in random.sample(range(len(train_dataset)), 2):
    print(f"Sample {index} of the training set: {train_dataset[index]}.")
    print(f"Decoded input_ids: {tokenizer.decode(train_dataset[index]['input_ids'])}")
    print(f"Decoded labels: {tokenizer.decode([label for label in train_dataset[index]['labels'] if label != -100])}")
    print("\n")


Sample 1034 of the training set: {'input_ids': tensor([    1,   682,   326,   508,   434,   326,  9395,   716, 14929,  1898,
         2353,  1245,  7353,   938,    18,     2,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,   

In [24]:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=batch_size
)

eval_dataloader = DataLoader(
    eval_dataset, shuffle=False, batch_size=batch_size
)

In [13]:
type(train_dataset['attention_mask'][0])

torch.Tensor

In [14]:
next(iter(train_dataloader))['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [13]:
import torch.nn as nn

class CodeT5_NLSQL(nn.Module):

  def __init__(self, model):
    super().__init__()

    self.model = model
    #self.input_size = model.config.to_dict()['hidden_size']
    #self.num_classes = num_classes

    #self.input_layer = nn.Linear(self.input_size)

  def forward(self, input_ids, attention_mask, labels=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    return outputs

In [14]:
import torch

In [40]:
nlsql_model = CodeT5_NLSQL(model)
nlsql_model.to(device)

CodeT5_NLSQL(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32100, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32100, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Linear(in_featur

In [16]:
import tqdm

In [17]:
dataset['validation']

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})

In [18]:
from tqdm.auto import tqdm

gold_file = open("gold.txt", "w")

gold_queries = []

for row in dataset['validation']:
    gold_queries.append(row['query'] + '\t' + row['db_id'])
    
gold_file.write("\n".join(gold_queries))
  
gold_file.close()

In [42]:
import numpy as np 
#from evaluation import evaluate, build_foreign_key_map_from_json
from evaluation2 import evaluate, build_foreign_key_map_from_json

def evaluate_model(model, dataloader, tokenizer, max_seq_length, device):
  model.eval()

  all_preds = []
  all_labels = []

  avg_batch_acc = 0
  pred_file = open("pred.txt", "w")

  with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluation"):
      input_ids = batch["input_ids"].to(device)
      labels = batch["labels"].to(device)

      attention_mask = batch["attention_mask"].to(device)
      #token_type_ids = batch["token_type_ids"].to(device)
      
      generated_tokens = model.model.generate(
            input_ids,
            max_length=max_seq_length,
            # beam_size=beam_size,
        )


      #logits = model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)

      #preds = torch.argmax(logits, dim=-1)
      #metric.add_batch(predictions=preds, references=labels)
      decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      
      # for row in decoded_preds:
      #   all_preds.append(row)

      labels = labels.tolist()

  
      for row in decoded_preds:
          all_preds.append("".join(row))
      

      new_labels = []
      for label_row in labels:
        if ' ' in label_row:
          print("we found a space")
        new_labels.append([value for value in label_row if value != -100])
        
      decoded_labels = tokenizer.batch_decode(new_labels, skip_special_tokens=True)
      break
    
  pred_file.write("\n".join(all_preds))


  pred_file.close()
  without_vals_scores = evaluate('gold.txt', 'pred.txt', 'database', 'match', build_foreign_key_map_from_json('tables.json'), False, False, False)
  with_vals_scores = evaluate('gold.txt', 'pred.txt', 'database', 'match', build_foreign_key_map_from_json('tables.json'), True, False, False)
  evaluation_results = {
      "eval/exact_match": without_vals_scores['all']['exact'],
      "eval/exact_match(vals)": with_vals_scores['all']['exact']
  }



  model.train()
  return evaluation_results, input_ids, decoded_preds, decoded_labels


In [20]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)


lr_scheduler = transformers.get_scheduler(
    name=lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=max_train_steps,
)

In [43]:
run = wandb.init(project=f"CODET5_SQLNL")

global_step = 0

progress_bar = tqdm(range(len(train_dataloader) * num_train_epochs))


# iterate over epochs
for epoch in range(num_train_epochs):
    nlsql_model.train()  # make sure that model is in training mode, e.g. dropout is enabled

    # iterate over batches
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        # decoder_input_ids = batch["decoder_input_ids"].to(device)
        # key_padding_mask = batch["encoder_padding_mask"].to(device)
        labels = batch["labels"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = nlsql_model(
            input_ids=input_ids,
            labels=labels,
            attention_mask=attention_mask
        )

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        global_step += 1

        wandb.log(
            {
                "train_loss": loss,
                "learning_rate": optimizer.param_groups[0]["lr"],
                "epoch": epoch,
            },
            step=global_step,
        )


        if global_step % eval_every_step == 0:
            eval_results, last_input_ids, last_decoded_preds, last_decoded_labels = evaluate_model(
                model=nlsql_model,
                dataloader=eval_dataloader,
                tokenizer=tokenizer,
                device=device,
                max_seq_length=max_seq_length,

            )    
            wandb.log(
              {"eval/exact_match": eval_results['eval/exact_match'], 
               "eval/exact_match(vals)": eval_results["eval/exact_match(vals)"]},
              step=global_step,
            )
            print("Generation example:")
            random_index = random.randint(0, len(last_input_ids) - 1)
            print(f"Input sentence: {tokenizer.decode(last_input_ids[random_index], skip_special_tokens=True)}")
            print(f"Generated sentence: {last_decoded_preds[random_index]}")
            print(f"Reference sentence: {last_decoded_labels[random_index]}")


        if global_step % logging_steps == 0:
            # An extra training metric that might be useful for understanding
            # how well the model is doing on the training set.
            # Please pay attention to it during training.
            # If the metric is significantly below 80%, there is a chance of a bug somewhere.
            predictions = logits.argmax(-1)

            label_nonpad_mask = labels != tokenizer.pad_token_id
            num_words_in_batch = label_nonpad_mask.sum().item()

            accuracy = (predictions == labels).masked_select(label_nonpad_mask).sum().item() / num_words_in_batch

            wandb.log(
                {"train_batch_word_accuracy": accuracy},
                step=global_step,
            )
            
            # print("Generation example:")
            # random_index = random.randint(0, len(last_input_ids) - 1)
            # print(f"Input sentence: {tokenizer.decode(last_input_ids[random_index], skip_special_tokens=True)}")
            # print(f"Generated sentence: {last_decoded_preds[random_index]}")
            # print(f"Reference sentence: {last_decoded_labels[random_index][0]}")

        # if global_step % args.eval_every_steps == 0 or global_step == args.max_train_steps:
        #     eval_results, last_input_ids, last_decoded_preds, last_decoded_labels = evaluate_model(
        #         model=model,
        #         dataloader=eval_dataloader,
        #         target_tokenizer=target_tokenizer,
        #         device=args.device,
        #         max_seq_length=args.max_seq_length,
        #         generation_type=args.generation_type,
        #         beam_size=args.beam_size,
        #         max_eval_steps=args.max_eval_steps
        #     )
        #     # YOUR CODE ENDS HERE
        #     wandb.log(
        #         {
        #             "eval/bleu": eval_results["bleu"],
        #             "eval/generation_length": eval_results["generation_length"],
        #         },
        #         step=global_step,
        #     )
        #     logger.info("Generation example:")
        #     random_index = random.randint(0, len(last_input_ids) - 1)
        #     logger.info(f"Input sentence: {source_tokenizer.decode(last_input_ids[random_index], skip_special_tokens=True)}")
        #     logger.info(f"Generated sentence: {last_decoded_preds[random_index]}")
        #     logger.info(f"Reference sentence: {last_decoded_labels[random_index][0]}")


        #     logger.info("Saving model checkpoint to %s", args.output_dir)
        #     model.save_pretrained(args.output_dir)

        # if global_step >= args.max_train_steps:
        #     break

run.finish()  # stop wandb run


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/exact_match,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
train_batch_word_accuracy,▁
train_loss,█▆█▅▅▄▃▄▃▁▄▅▄▃▃▅▂▂▂▄▁▃▂▃▂▁
epoch,0
eval/exact_match,0.0
learning_rate,1e-05
train_batch_word_accuracy,0.07812
train_loss,4.06914


Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]


easy pred: I am
easy gold: SELECT count(*) FROM singer

easy pred: Total singers?
easy gold: SELECT count(*) FROM singer

easy pred: I am
easy gold: SELECT count(*) FROM singer

easy pred: Total singers?
easy gold: SELECT count(*) FROM singer

Generation example:
Input sentence: How many singers do we have?
Generated sentence: I am
Reference sentence: SELECT count(*) FROM singer


Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]


easy pred: do we have?
easy gold: SELECT count(*) FROM singer

easy pred: Total singers?
easy gold: SELECT count(*) FROM singer

easy pred: do we have?
easy gold: SELECT count(*) FROM singer

easy pred: Total singers?
easy gold: SELECT count(*) FROM singer

Generation example:
Input sentence: How many singers do we have?
Generated sentence: do we have?
Reference sentence: SELECT count(*) FROM singer


...

Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]


easy pred: How many singers we have?
easy gold: SELECT count(*) FROM singer

easy pred: Total singers:
easy gold: SELECT count(*) FROM singer

easy pred: How many singers we have?
easy gold: SELECT count(*) FROM singer

easy pred: Total singers:
easy gold: SELECT count(*) FROM singer

Generation example:
Input sentence: What is the total number of singers?
Generated sentence: Total singers:
Reference sentence: SELECT count(*) FROM singer


Evaluation:   0%|          | 0/517 [00:01<?, ?it/s]


easy pred: How many singers we have?
easy gold: SELECT count(*) FROM singer

easy pred: Total singers:
easy gold: SELECT count(*) FROM singer

easy pred: How many singers we have?
easy gold: SELECT count(*) FROM singer

easy pred: Total singers:
easy gold: SELECT count(*) FROM singer

Generation example:
Input sentence: How many singers do we have?
Generated sentence: How many singers we have?
Reference sentence: SELECT count(*) FROM singer


KeyboardInterrupt: 